# Algebraic Types

Besides inductively defined lists, we can define other types of data. An example is the binary tree type.

## Binary Trees
A binary tree is a data type that can be defined inductively as:

In [ ]:
type 'a tree = Empty | Node of 'a * 'a tree * 'a tree

An example of a binary tree with the following structure is:

```ocaml
    2
   / \
  1   3 
```


In [ ]:
let t0 = Node(2, 
              Node(1, 
                   Empty, 
                   Empty), 
              Node(3, 
                   Empty, 
                   Empty))

To insert an element into a binary tree in OCaml, we generate the binary tree that results from inserting the element. We do this inductively on the structure of the tree. There are many ways to insert an element into a binary tree. One of them is the following:


In [ ]:
let rec insert x t = 
  match t with 
  | Empty -> Node(x, Empty, Empty)
  | Node(y,l,r) -> Node(x,t,Empty)

let _ = assert (insert 0 t0 = Node (0, Node (2, Node (1, Empty, Empty), Node (3, Empty, Empty)), Empty))



## Binary Search Trees 
A binary search tree is a binary tree with the following property: for each node, the value of the node is greater than all the values in the left subtree and smaller than all the values in the right subtree. Additionally, the left subtree and the right subtree are also binary search trees.

The insertion of an element into a binary search tree, while preserving the binary search tree property, is done as follows:

In [ ]:

let rec insert_bst x t = 
  match t with 
  | Empty -> Node(x, Empty, Empty)
  | Node(y,l,r) -> if x < y then Node(y,insert_bst x l,r) else Node(y,l, insert_bst x r)

let t1 = insert_bst 0 t0
let t2 = Node(4, t0, Empty)
let _ = assert(t1 = Node (2, Node (1, Node (0, Empty, Empty), Empty), Node (3, Empty, Empty)))


Let's define an auxiliary function `remove_min` that removes the node with the smallest value from a binary search tree. This function will be useful for removing a node from the binary search tree. The `remove_min` function will return the binary search tree without the node containing the smallest value.

In [ ]:
(** [remove_min t] denotes the tree removing the minimum value from [t]
    pre: [t <> Empty] *)
let rec remove_min t = 
  match t with 
  | Empty -> failwith "Precondition violated!"
  | Node(x,Empty,r) -> r
  | Node(x,l,r) -> Node(x, remove_min l, r)

(** [min_tree t] denotes the smaller value in [t]
    pre: [t <> Empty] *)
let rec min_tree t = 
  match t with 
  | Empty -> failwith "Precondition violated!"
  | Node(x,Empty,r) -> x
  | Node(x,l,r) -> min_tree l

(** [min_tree t] denotes the larger value in [t]
    pre: [t <> Empty] *)
let rec max_tree t = 
  match t with 
  | Empty -> failwith "Precondition violated!"
  | Node(x,_,Empty) -> x
  | Node(_,_,r) -> max_tree r

Given these auxiliary functions, we can define a function that corresponds to the inductive proof of the binary search tree property. If we consider that the recursive call on the left subtree and on the right subtree corresponds to the induction hypothesis (the property for a smaller tree), then we can define the result for the composed tree by checking whether all elements in the left subtree are smaller than the root and all elements in the right subtree are greater than the root.

In [ ]:
let rec is_a_bst t = 
  match t with 
  (* Base case *)
  | Empty -> true  

  (* Base case *)
  | Node(_, Empty,Empty) -> true

  (* Inductive case if the left subtree is empty *)
  | Node(x,Empty,r) -> (* by h.i. *) is_a_bst r && min_tree r >= x

  (* Inductive case if the right subtree is empty *)
  | Node(x,l,Empty) -> (* by h.i. *) is_a_bst l && max_tree l < x 

  (* General Inductive case *)
  | Node(x,l,r) -> (* by h.i. *) is_a_bst l && (* by h.i. *) is_a_bst r && max_tree l < x && min_tree r >= x

let _ = assert(is_a_bst t0)
let _ = assert(is_a_bst t1)
let _ = assert(is_a_bst t2)


The removal of a value from a binary search tree is done by searching for the value and removing it. When an intermediate node is removed, it must be replaced by the minimum value of the right subtree (or by the maximum value of the left subtree). This is done by calling the `remove_min` function on the right subtree and replacing the intermediate node with the result. The left subtree remains unchanged. The result is a new binary search tree with the same properties as before, but without the removed value.

In [ ]:
let rec remove x t = 
  match t with
  | Empty -> Empty
  | Node(y,l,r) -> 
    if x = y then 
      Node(min_tree r, l, remove_min r)
    else 
      if x < y then 
        Node(y, remove x l ,r) 
      else 
        Node(y, l, remove x r)

let _ = assert (remove 2 t0 = Node (3, Node (1, Empty, Empty), Empty))
let _ = assert (remove 2 t2 = Node (4, Node (3, Node (1, Empty, Empty), Empty), Empty))

- binary trees
- insert in a bst
- remove in a bst (remove_min)
- tree_size
- depth
- pre_order 
- in_order
- post_order
- map
- fold_in_order
- fold_pre_order
- fold_post_order

An alternative to the `remove_min` function above is to compute the minimum value at the same time as it is removed. The `remove` function is then defined as follows:

In [ ]:
(** pre: [t <> Empty] *)
let rec remove_min t = 
  match t with 
  | Empty -> failwith "There is no minimum"
  | Node(y,Empty,r) -> (y,r)
  | Node(y,l,r) -> let (min,l') = remove_min l in (min, Node(y,l',r))

let rec remove x t = 
  match t with 
  | Empty -> Empty
  | Node(y,l,r) -> 
      if x = y then 
        begin match r with
        | Empty -> l
        | _ -> let (min,r') = remove_min r in Node(min, l, r')
        end
      else if x < y then 
        Node(y, remove x l, r)
      else 
        Node(y, l, remove x r)

let _ = assert (remove 2 t0 = Node (3, Node (1, Empty, Empty), Empty))
let _ = assert (remove 2 t2 = Node (4, Node (3, Node (1, Empty, Empty), Empty), Empty))

Other functions on trees that we can define are the following:

In [ ]:
let rec tree_size t = 
  match t with
  | Empty -> 0
  | Node(_, l, r) -> 1 + tree_size l + tree_size r

  
let rec tree_depth t = 
  match t with
  | Empty -> 0
  | Node(_,l,r) -> 1 + max (tree_depth l) (tree_depth r)
    
let _ = assert (4 = tree_size t2)
let _ = assert (3 = tree_depth t2)


# Traversals in Binary Trees

Traversals in binary trees are functions that traverse the tree in a specific way. There are three types of binary tree traversals:

- **pre-order**: visits the root, then the left subtree, and then the right subtree
- **in-order**: visits the left subtree, then the root, and then the right subtree
- **post-order**: visits the left subtree, then the right subtree, and then the root


In [ ]:
let t = Node(1,Empty,Node(2,Empty,Node (3,Empty,Empty)))
let t3 = Node(0, t, t)

let rec pre_order t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> x::(pre_order l)@(pre_order r)

let _ = assert([0;1;2;3;1;2;3] = pre_order t3)

In [ ]:
let rec in_order t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> (in_order l)@[x]@(in_order r)

let _ = assert( [1;2;3;0;1;2;3] = in_order t3)

In [ ]:
let ot = Node(4,Node(2,Node(1,Empty,Empty),Node(3,Empty,Empty)),Node(6,Node(5,Empty,Empty),Node(7,Empty,Empty)))

let _ = assert( [1; 2; 3; 4; 5; 6; 7] = in_order ot)
let _ = assert( [4; 2; 1; 3; 6; 5; 7] = pre_order ot)

In [ ]:
let rec post_order t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> (post_order l)@(post_order r)@[x]

let _ = assert( [1; 3; 2; 5; 7; 6; 4] = post_order ot)

In [ ]:
let rec in_order_rev t = 
  match t with 
  | Empty -> []
  | Node(x,l,r) -> (in_order_rev r)@[x]@(in_order_rev l)

let _ = assert( [7; 6; 5; 4; 3; 2; 1] = in_order_rev ot)

# Higher-order functions (Map e Fold)

In [ ]:
let rec map f t = 
  match t with
  | Empty -> Empty
  | Node(x,l,r) -> Node(f x, map f l, map f r)

let _ = map string_of_int ot
let _ = map ((+)1) ot

In [ ]:
let map_opt f o = 
  match o with
  | None -> None
  | Some v -> Some (f v)

let _ = map_opt string_of_int (Some 1)
let _ = map_opt string_of_int None
let _ = map_opt ((+)1) (Some 1)
let _ = map_opt ((+)1) None

In [ ]:
let rec fold_left f acc l = 
  match l with 
  | [] -> acc
  | x::xs -> fold_left f (f acc x) xs

let _ = fold_left (+) 0 (pre_order ot)

let rec fold_in_order f acc t =  
  match t with
  | Empty -> acc
  | Node(x,l,r) -> let acc_l = fold_in_order f acc l in let acc_root = f acc_l x in fold_in_order f acc_root r

let _ = fold_in_order (+) 0 ot
let _ = fold_in_order (fun acc x -> acc@[x]) [] ot
let filter p t = fold_in_order (fun acc x -> if p x then acc@[x] else acc) [] ot

let _ = filter (fun x -> x mod 2 = 0) ot



In [ ]:
let rec fold_pre_order f acc t =  
  match t with
  | Empty -> acc
  | Node(x,l,r) -> let acc_root = f acc x in let acc_l = fold_pre_order f acc_root l in fold_pre_order f acc_l r

let _ = fold_pre_order (fun acc x -> acc@[x]) [] ot
